We will implement the code examples for:
* sktime: For research-grade rigor and composable pipelines.
* darts: For rapid prototyping and ease of use.
* mlforecast: For high-performance scaling using Numba.
* autogluon: For automated benchmarking and ensembling.
* skforecast: For direct integration with scikit-learn regressors.

# Sktime

In [5]:
from sktime.forecasting.model_selection import ForecastingGridSearchCV, SlidingWindowSplitter
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.ets import AutoETS
from sktime.datasets import load_airline
import numpy as np

# 1. Load data in the pd.Series format sktime expects
y = load_airline()
# 2. Split into training and validation (e.g., last 12 months as validation)
y_train, y_val = temporal_train_test_split(y, test_size=12)

# 2. Define the forecaster (our model)
# The API feels very similar to scikit-learn
forecaster = AutoETS()

# 3. Define the temporal cross-validation strategy from Chapter 3
cv = SlidingWindowSplitter(window_length=50, step_length=12, fh=1)

# 4. Define the parameter grid to search
param_grid = {"error": ["add", "mul"], "trend": ["add", "mul"]}

# 5. Set up and run the grid search
# This combines the model, CV, and params into a single, robust workflow
gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    cv=cv,
    param_grid=param_grid,
    n_jobs=-1
)

gscv.fit(y_train)
print(f"Best parameters found: {gscv.best_params_}")

# 6. Forecast the validation horizon
fh = list(range(1, len(y_val)+1))  # forecast horizon = length of validation
y_pred = gscv.predict(fh=fh)

# 7. Compute performance (MAPE)
mape = mean_absolute_percentage_error(y_val, y_pred) * 100

# 8. Show first 5 predictions like your previous output
prediction_preview = pd.DataFrame(y_pred).head()
prediction_preview.columns = ["#Passengers"]
print("Prediction (first 5 values):")
print(prediction_preview)
print(f"\nMAPE on validation set: {mape:.2f}%")

/Users/ben/anaconda3/envs/ml_timeseries/lib/python3.12/site-packages/sktime/forecasting/model_selection/_base.py:76: UserWarning: Parameter n_jobs of ForecastingGridSearchCV has been removed in sktime 0.27.0 and is no longer used. It is ignored when passed. Instead, the backend and backend_params parameters should be used to pass n_jobs or other parallelization parameters.
  warn(


Best parameters found: {'error': 'add', 'trend': 'mul'}
Prediction (first 5 values):
         #Passengers
1960-01   466.578473
1960-02   470.358248
1960-03   474.168644
1960-04   478.009907
1960-05   481.882289

MAPE on validation set: 13.23%


# Darts

In [2]:
from darts.datasets import AirPassengersDataset
from darts.models import ExponentialSmoothing
from darts import TimeSeries
from darts.metrics import mape

# 1. Load data into a TimeSeries object, darts' standard container
series: TimeSeries = AirPassengersDataset().load()

# 2. Split the data, holding out the last 36 months for validation
train, val = series[:-36], series[-36:]

# 3. Create and train the model using the unified API
# The code remains the same even if you swap ExponentialSmoothing with a deep learning model
model = ExponentialSmoothing()
model.fit(train)

# 4. Predict the next len(val) timesteps
prediction = model.predict(len(val))

# 5. Inspect the forecast and evaluate its performance
print("Prediction (first 5 values):")
print(prediction[:5])
print(f"MAPE on validation set: {mape(val, prediction):.2f}%")

/Users/ben/anaconda3/envs/ml_timeseries/lib/python3.12/site-packages/threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Prediction (first 5 values):
            #Passengers
Month                  
1958-01-01   357.332132
1958-02-01   345.819810
1958-03-01   398.634260
1958-04-01   390.191635
1958-05-01   396.345978

shape: (5, 1, 1), freq: MS, size: 40.00 B
MAPE on validation set: 5.11%


# mlforecast

In [10]:
import pandas as pd
import numpy as np
from mlforecast import MLForecast
from mlforecast.lag_transforms import RollingMean
from mlforecast.utils import generate_daily_series
import lightgbm as lgb
from sklearn.metrics import mean_absolute_percentage_error

# 1. Generate synthetic daily series
series_df = generate_daily_series(n_series=10, min_length=100, max_length=150)

# 2. Split into train/validation (last 14 days for validation)
h = 14
train_df = series_df.groupby("unique_id", group_keys=False, observed=True).apply(lambda x: x.iloc[:-h])
val_df = series_df.groupby("unique_id", group_keys=False, observed=True).apply(lambda x: x.iloc[-h:])


# 3. Define LightGBM model
model = lgb.LGBMRegressor(
    random_state=42,
    n_jobs=1,
    num_leaves=64,
    min_child_samples=5,
    learning_rate=0.05,
    verbose=-1,
    force_row_wise=True
)

# 4. Define MLForecast
fcst = MLForecast(
    models={'lgbm': model},
    freq='D',
    lags=[1,7,14],
    lag_transforms={1: [RollingMean(window_size=7, min_samples=1)]},
    date_features=['dayofweek','month'],
    num_threads=1
)

# 5. Fit on training data
fcst.fit(train_df)

# 6. Predict validation horizon
y_pred = fcst.predict(h=h)

# 7. Merge predictions with validation
merged = val_df.merge(y_pred, on=['unique_id','ds'])
merged = merged.rename(columns={'y': 'y_true'})  # rename target column for clarity

# 8. Compute per-series and overall MAPE
merged['ape'] = np.abs(merged['y_true'] - merged['lgbm']) / merged['y_true']
mape_per_series = merged.groupby('unique_id')['ape'].mean() * 100
overall_mape = merged['ape'].mean() * 100

# 9. Show first 5 predictions
prediction_preview = y_pred.head()
print("Prediction (first 5 values):")
print(prediction_preview)

print(f"\nMAPE per series:\n{mape_per_series}")
print(f"\nOverall MAPE: {overall_mape:.2f}%")


Prediction (first 5 values):
  unique_id         ds      lgbm
0      id_0 2000-05-10  4.256834
1      id_0 2000-05-11  5.278685
2      id_0 2000-05-12  6.199548
3      id_0 2000-05-13  0.173573
4      id_0 2000-05-14  1.253689

MAPE per series:
unique_id
id_0     51.423117
id_1     21.437474
id_2      8.048068
id_3      9.251338
id_4     24.030832
id_5     11.027903
id_6      5.535145
id_7      9.775673
id_8      7.743888
id_9    109.184316
Name: ape, dtype: float64

Overall MAPE: 25.75%


/var/folders/zs/dpjs6jdx52xgnf97mcsn19240000gp/T/ipykernel_26706/2158128676.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = series_df.groupby("unique_id", group_keys=False, observed=True).apply(lambda x: x.iloc[:-h])
/var/folders/zs/dpjs6jdx52xgnf97mcsn19240000gp/T/ipykernel_26706/2158128676.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_df = series_df.groupby("unique_id", group_keys=False, 

# Autogluon

In [1]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
import numpy as np

# 1. Sample data
ids = ["A", "B", "C"]
timestamps = pd.to_datetime(pd.date_range("2023-01-01", periods=100))
data = []
for item_id in ids:
    target = np.random.randn(100).cumsum()
    data.append(pd.DataFrame({"item_id": item_id, "timestamp": timestamps, "target": target}))

df = pd.concat(data)

# 2. Convert to AutoGluon format
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp"
)

# 3. Fit a predictor with only truly lightweight models
predictor = TimeSeriesPredictor(
    prediction_length=24,
    target="target",
    eval_metric="MASE",
    verbosity=2
)

# Only train models guaranteed to be very light
predictor.fit(
    train_data,
    hyperparameters={
        "Naive": {},
        "SeasonalNaive": {}
    },
    num_val_windows=1,       # keep 1 validation window
    refit_every_n_windows=None  # avoid repeated refitting, saves memory
)

# 4. Leaderboard
print("Model performance leaderboard:")
print(predictor.leaderboard())

Beginning AutoGluon training...
AutoGluon will save models to '/Users/ben/Machine-Learning-for-Time-Series-with-Python-Second-Edition/chapter4/AutogluonModels/ag-20251206_130136'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.6.0: Fri Nov 15 17:21:49 PST 2024; root:xnu-8796.141.3.709.7~2/RELEASE_X86_64
CPU Count:          4
GPU Count:          0
Memory Avail:       4.69 GB / 16.00 GB (29.3%)
Disk Space Avail:   202.49 GB / 465.63 GB (43.5%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'hyperparameters': {'Naive': {}, 'SeasonalNaive': {}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 24,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferre

Model performance leaderboard:
              model  score_val  pred_time_val  fit_time_marginal  fit_order
0  WeightedEnsemble  -1.028021       6.392401           0.313400          3
1             Naive  -1.028021       6.392401           0.029221          1
2     SeasonalNaive  -1.470471       0.032117           0.020691          2


# Skforecast

In [4]:
from skforecast.recursive import ForecasterRecursive
from sklearn.ensemble import RandomForestRegressor
from sktime.datasets import load_airline
import pandas as pd

# 1. Load data as a standard pandas Series
y = load_airline()
y.index = pd.to_datetime(y.index.to_timestamp()) # Ensure datetime index

# 2. Create the scikit-learn model you want to use
regressor = RandomForestRegressor(random_state=42)

# 3. Create the forecaster, passing the regressor and specifying lags
forecaster = ForecasterRecursive(
    estimator=regressor,
    lags=15  # Create 15 lag features
)

# 4. Fit the forecaster on the training data
forecaster.fit(y=y[:-36])

# 5. Predict the next 36 steps
predictions = forecaster.predict(steps=36)

print("Forecasts (first 5 values):")
print(predictions.head())

Forecasts (first 5 values):
1958-01-01    352.06
1958-02-01    350.19
1958-03-01    422.17
1958-04-01    431.52
1958-05-01    429.42
Freq: MS, Name: pred, dtype: float64
